In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
patients_df = pd.read_csv('all_data/patients.csv')
admissions_df = pd.read_csv('all_data/admissions.csv')
diagnosis_df = pd.read_csv('all_data/diagnoses_icd.csv.gz',skiprows=1,header=None)
headerv = pd.read_csv('all_data/diagnoses_icd.csv.gz')

## Explore patients.csv file and visualize the data
### Exploring male vs female counts

In [ ]:
x_gender = ['Male','Female']
y_gender = [len(patients_df[patients_df['gender'] == 'M']),len(patients_df[patients_df['gender'] == 'M'])]

plt.bar(x_gender,y_gender)
plt.title('Male vs Female counts')
plt.xlabel('Gender')
plt.ylabel('Counts')
plt.show()

### Exploring anchor for age

In [ ]:
x_age = ['2011 - 2013','2014 - 2016']
y_age = [len(patients_df[patients_df['anchor_year_group'] == '2011 - 2013']),len(patients_df[patients_df['anchor_year_group'] == '2014 - 2016'])]

plt.bar(x_age,y_age)
plt.title('Anchor for age')
plt.xlabel('Age Bracket')
plt.ylabel('Counts')
plt.show()

## Visualize admissions.csv file
The csv file has dates that have been deidentified and hence (might!) not be useful for this purpose

### Admission type count

In [ ]:

y_admission_type = admissions_df['admission_type'].value_counts()

plt.figure(figsize=(6,4))
y_admission_type.plot(kind='bar')


plt.title('Count of admission type')
plt.xlabel('Admission Type')
plt.ylabel('Count')
plt.show()

### Admission Location Count

In [ ]:
y_admission_location = admissions_df['admission_location'].value_counts()

plt.figure(figsize=(6,4))
y_admission_location.plot(kind='bar')


plt.title('Count of admission location')
plt.xlabel('Admission Location')
plt.ylabel('Count')
plt.show()

### Discharge Location

In [ ]:
y_discharge_location = admissions_df['discharge_location'].value_counts()

plt.figure(figsize=(6,4))
y_discharge_location.plot(kind='bar')


plt.title('Count of discharge location')
plt.xlabel('Discharge Location')
plt.ylabel('Count')
plt.show()

As we can see from the above graph, there are two discharge locations : 'HOME' & 'HOME HEALTH CARE'

### Treatment Effectiveness at the hospital

In [ ]:
hospital_expire_flag = admissions_df['hospital_expire_flag'].value_counts()

plt.figure(figsize=(6,4))
hospital_expire_flag.plot(kind='bar')


plt.title('Successful treatment vs deaths')
plt.xlabel('Successful treatment vs deaths')
plt.ylabel('')
plt.show()

### Duration of stay at the hospital:

Dividing the date ranges into the following anchors:
1. less than 1 day - 120938 patients
2. 1-3 days - 169187 patients
3. 3-10 days - 199954 patients
4. 10-30 days - 48803 patients
5. more than 30 days - 7146 patients

In [ ]:
import datetime
def duration(date1,date2):
    return abs(date2-date1)

def convert(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    datetime_str = datetime.datetime.strptime(date_time, format)
    return datetime_str



admission_duration = []

for row in admissions_df.iterrows():
    # print(row[1])
    # print(row[1].admittime)
    date1 = convert(row[1].admittime)
    date2 = convert(row[1].dischtime)
    admission_duration.append((duration(date1,date2)
                                       ))

series = pd.Series([td.total_seconds() / 86400 for td in admission_duration])

# Define bins and labels
bins = [-1, 1, 3, 10, 30, float("inf")]
labels = ["less than 1 day", "1-3 days", "3-10 days", "10-30 days", "more than 30 days"]

# Categorize the series
categories = pd.cut(series, bins=bins, labels=labels)

# Count occurrences per category
duration_counts = categories.value_counts().sort_index()


plt.figure(figsize=(6,4))
duration_counts.plot(kind='bar')


plt.title('Duration of stay of patients in the hospital')
plt.xlabel('Day Anchors')
plt.ylabel('Count of Patients')
plt.show()

## Building metadata

### Now that I have a rough idea of the data and entities I am working with, I will begin by creating the vocabulary

In [ ]:
# Installing the dependencies

!pip install rdflib

In [ ]:
import rdflib
import csv
import requests

from    rdflib import Graph, Namespace, URIRef, Literal
from    rdflib.namespace import XSD, RDF, RDFS, SDO
from datetime import datetime
import pandas as pd
import pandas as pd
from IPython.display import display, HTML
from pygments import highlight
from pygments.lexers import SparqlLexer
from pygments.formatters import HtmlFormatter
from rdflib import Graph

In [ ]:
g = Graph()

BM = rdflib.Namespace('https://biomedit.ch/rdf/sphn-schema/sphn#')     # BM -> SPHN URI
OWL = rdflib.Namespace("http://www.w3.org/2002/07/owl#")
MESH = rdflib.Namespace("http://purl.bioontology.org/ontology/MESH/")  # MESH -> Medical Subject Heading URI
PU = rdflib.Namespace("http://example.org/PU/")                        # PU -> Patient URI 
ANU = rdflib.Namespace("http://example.org/AN/")                       # ANU -> Admission Node URI
DU = rdflib.Namespace("http://example.org/DU/")                        # DU -> Drug URI
LU = rdflib.Namespace("http://example.org/LU/")                        # LU -> Lab Test URI
DIAGU = rdflib.Namespace("http://example.org/DIAGU/")                  # DIAGU -> Diagnosis URI

g.bind('bm',BM)
g.bind('xsd',XSD)
g.bind('rdf',RDF)
g.bind('rdfs',RDFS)
g.bind('sdo',SDO)
g.bind('mesh',MESH)
g.bind('owl',OWL)
g.bind("pu",PU)
g.bind("anu",ANU)
g.bind("du",DU)
g.bind("lu",LU)
g.bind("diagu",DIAGU)

## Defining Schema of the Knowledge Graph

In [ ]:
# Custom Classes
g.add((SDO.Event,RDF.type,RDFS.Class))
g.add((SDO.Person,RDF.type,RDFS.Class))
g.add((SDO.Patient,RDF.type,SDO.Class))
g.add((SDO.MedicalAudience,RDF.type,SDO.Class))
g.add((SDO.Audience,RDF.type,SDO.Class))
g.add((SDO.Thing,RDF.type,SDO.Class))
g.add((SDO.CreativeWork,RDF.type,SDO.Class))


# Establishing class hierarchy
g.add((SDO.Event,RDFS.subClassOf,SDO.Thing))
g.add((SDO.Patient,RDFS.subClassOf,SDO.Person))
g.add((SDO.Patient,RDFS.subClassOf,SDO.MedicalAudience))
g.add((SDO.MedicalAudience,RDFS.subClassOf,SDO.Audience))
g.add((SDO.Person,RDFS.subClassOf,SDO.Thing))
g.add((SDO.CreativeWork,RDFS.subClassOf,SDO.Thing))


# Declaring Properties
g.add((SDO.actor,RDF.type,RDF.Property))
g.add((SDO.actor,RDFS.domain,SDO.Event))
g.add((SDO.Event,OWL.equivalentClass,BM.AdministrativeCase))
g.add((SDO.actor,RDFS.range,SDO.Patient))
g.add((SDO.actor,RDFS.label,Literal("associatePatient",lang='en')))

g.add((SDO.workPerformed,RDF.type,RDF.Property))
g.add((SDO.workPerformed,RDFS.domain,SDO.event))
g.add((SDO.event,OWL.equivalentClass,BM.AdministrativeCase))      # Equivalency of classes between Schema Event Class and SPHN Administratve Case Class
g.add((SDO.workPerformed,RDFS.range,SDO.CreativeWork))
g.add((SDO.CreativeWork,OWL.equivalentClass,BM.LabTestEvent))     # Equivalency of classes between Schema Creative work Class and SPHN Lab Test Class
g.add((SDO.workPerformed,RDFS.label,Literal("labWorkPerformed",lang='en')))


g.add((SDO.gender,RDF.type,RDF.Property))
g.add((SDO.gender,RDFS.domain,SDO.Patient))
g.add((SDO.gender,RDFS.range,XSD.string))
g.add((SDO.gender,RDFS.label,Literal("gender",lang='en')))

g.add((SDO.audienceType,RDF.type,RDF.Property))
g.add((SDO.audienceType,RDFS.domain,SDO.Patient))
g.add((SDO.audienceType,RDFS.range,XSD.string))
g.add((SDO.audienceType,RDFS.label,Literal("ethnicity",lang='en')))

g.add((SDO.knowsLanguage,RDF.type,RDF.Property))
g.add((SDO.knowsLanguage,RDFS.domain,SDO.Patient))
g.add((SDO.knowsLanguage,RDFS.range,XSD.string))
g.add((SDO.knowsLanguage,RDFS.label,Literal("knowsLanguage",lang='en')))

g.add((SDO.additionalType,RDF.type,RDF.Property))
g.add((SDO.additionalType,RDFS.domain,SDO.Patient))
g.add((SDO.additionalType,RDFS.range,XSD.string))
g.add((SDO.additionalType,RDFS.label,Literal("admissionType",lang='en')))



In [ ]:


# Declaring the main classes
g.add((BM.SPHNConcept, RDF.type,RDFS.Class))
g.add((BM.AdministrativeCase, RDF.type, RDFS.Class))
g.add((BM.Admission,RDF.type,RDFS.Class))
g.add((BM.Discharge,RDF.type,RDFS.Class))
g.add((BM.Death,RDF.type,RDFS.Class))
g.add((BM.DeathDate,RDF.type,RDFS.Class))
g.add((BM.Location,RDF.type,RDFS.Class))
g.add((BM.InsuranceStatus,RDF.type,RDFS.Class))
g.add((BM.DrugAdministrationEvent,RDF.type,OWL.Class))
g.add((BM.Drug,RDF.type,OWL.Class))
g.add((BM.Substance,RDF.type,OWL.Class))
g.add((BM.LabTestEvent,RDF.type,OWL.Class))
g.add((BM.LabTest,RDF.type,OWL.Class))
g.add((BM.LabResult,RDF.type,OWL.Class))
g.add((BM.Quantity,RDF.type,OWL.Class))
g.add((BM.ReferenceValue,RDF.type,OWL.Class))
g.add((BM.ReferenceRange,RDF.type,OWL.Class))
g.add((BM.Sample,RDF.type,OWL.Class))


# Establishing subclasses relashionship
g.add((BM.AdministrativeCase,RDFS.subClassOf,BM.SPHNConcept))
       
# Declaring Properties
g.add((BM.hasAdmission,RDF.type,RDF.Property))
g.add((BM.hasAdmission,RDFS.domain,BM.AdministrativeCase))
g.add((BM.hasAdmission,RDFS.range,BM.Admission))
g.add((BM.hasAdmission,RDFS.label,Literal("hasAdmission",lang='en')))

g.add((BM.hasDischarge,RDF.type,RDF.Property))
g.add((BM.hasDischarge,RDFS.domain,BM.AdministrativeCase))
g.add((BM.hasDischarge,RDFS.range,BM.Discharge))
g.add((BM.hasDischarge,RDFS.label,Literal("hasDischarge",lang='en')))

g.add((BM.hasOriginLocation,RDF.type,RDF.Property))
g.add((BM.hasOriginLocation,RDFS.domain,BM.Admission))
g.add((BM.hasOriginLocation,RDFS.range,BM.Location))
g.add((BM.hasOriginLocation,RDFS.label,Literal("admissionOrigin",lang='en')))

g.add((BM.hasExact,RDF.type,RDF.Property))
g.add((BM.hasExact,RDFS.domain,BM.Location))
g.add((BM.hasExact,RDFS.range,XSD.string))
g.add((BM.hasExact,RDFS.label,Literal("exactLocation",lang='en')))

g.add((BM.hasTargetLocation,RDF.type,RDF.Property))
g.add((BM.hasTargetLocation,RDFS.domain,BM.Discharge))
g.add((BM.hasTargetLocation,RDFS.range,BM.Location))
g.add((BM.hasTargetLocation,RDFS.label,Literal("targetLocation",lang='en')))

g.add((BM.hasAdministrativeCase,RDF.type,RDF.Property))
g.add((BM.hasAdministrativeCase,RDFS.domain,BM.InsuranceStatus))
g.add((BM.hasAdministrativeCase,RDFS.range,BM.AdministrativeCase))
g.add((BM.hasAdministrativeCase,RDFS.label,Literal("insuranceCase",lang='en')))

g.add((BM.hasDateTime,RDF.type,RDF.Property))
g.add((BM.hasDateTime,RDFS.domain,BM.Admission))
g.add((BM.hasDateTime,RDFS.range,XSD.datetime))
g.add((BM.hasDateTime,RDFS.label,Literal("admitTime",lang='en')))

g.add((BM.hasDateTime,RDFS.domain,BM.Discharge))
g.add((BM.hasDateTime,RDFS.range,XSD.datetime))
g.add((BM.hasDateTime,RDFS.label,Literal("dischargeTime",lang='en')))

g.add((BM.hasCoverageType,RDF.type,RDF.Property))
g.add((BM.hasCoverageType,RDFS.domain,BM.InsuranceStatus))
g.add((BM.hasCoverageType,RDFS.range,XSD.string))
g.add((BM.hasCoverageType,RDFS.label,Literal("hasCoverageType",lang='en')))

# Defining Drug Administration Event Property
g.add((BM.hasStartDateTime,RDF.type,RDF.Property))
g.add((BM.hasStartDateTime,RDFS.domain,BM.DrugAdministrationEvent))
g.add((BM.hasStartDateTime,RDFS.range,XSD.datetime))
g.add((BM.hasStartDateTime,RDFS.label,Literal("drugStartTime",lang='en')))

g.add((BM.hasEndDateTime,RDF.type,RDF.Property))
g.add((BM.hasEndDateTime,RDFS.domain,BM.DrugAdministrationEvent))
g.add((BM.hasEndDateTime,RDFS.range,XSD.datetime))
g.add((BM.hasEndDateTime,RDFS.label,Literal("drugEndTime",lang='en')))

g.add((BM.hasAdministrativeCase,RDF.type,RDF.Property))
g.add((BM.hasAdministrativeCase,RDFS.domain,BM.DrugAdministrationEvent))
g.add((BM.hasAdministrativeCase,RDFS.range,BM.AdministrativeCase))
g.add((BM.hasAdministrativeCase,RDFS.label,Literal("hasDrugAdministrativeCase",lang='en')))

g.add((BM.hasDrug,RDF.type,RDF.Property))
g.add((BM.hasDrug,RDFS.domain,BM.DrugAdministrationEvent))
g.add((BM.hasDrug,RDFS.range,BM.Drug))    # Add an equivalent class
g.add((BM.hasDrug,RDFS.label,Literal("hasDrugAdministrativeCase",lang='en')))

# Defining Drug Property
g.add((BM.hasActiveIngredient,RDF.type,RDF.Property))
g.add((BM.hasActiveIngredient,RDFS.domain,BM.Drug))
g.add((BM.hasActiveIngredient,RDFS.range,BM.Substance))
g.add((BM.hasActiveIngredient,RDFS.label,Literal("hasActiveIngredient",lang='en')))

# Defining Substance Property
g.add((BM.hasCode,RDF.type,RDF.Property))
g.add((BM.hasCode,RDFS.domain,BM.Substance))
g.add((BM.hasCode,RDFS.range, XSD.String))
g.add((BM.hasCode,RDFS.label,Literal("hasCode",lang='en')))

g.add((BM.hasGenericName,RDF.type,RDF.Property))
g.add((BM.hasGenericName,RDFS.domain,BM.Substance))
g.add((BM.hasGenericName,RDFS.range, XSD.String))
g.add((BM.hasGenericName,RDFS.label,Literal("hasCode",lang='en')))

g.add((BM.hasValue,RDF.type,RDF.Property))
g.add((BM.hasValue,RDFS.domain,BM.Quantity))
g.add((BM.hasValue,RDFS.range,XSD.Float))
g.add((BM.hasValue,RDFS.label,Literal("hasValue",lang='en')))

g.add((BM.hasLabTest,RDF.type,RDF.Property))
g.add((BM.hasLabTest,RDFS.domain,BM.LabTestEvent))
g.add((BM.hasLabTest,RDFS.range,BM.LabTest))
g.add((BM.hasLabTest,RDFS.label,Literal("hasLabTest",lang='en')))

g.add((BM.hasDateTime,RDF.type,RDF.Property))
g.add((BM.hasDateTime,RDFS.domain,BM.LabTestEvent))
g.add((BM.hasDateTime,RDFS.range,XSD.datetime))
g.add((BM.hasDateTime,RDFS.label,Literal("hasDateTime",lang='en')))

g.add((BM.hasAdministrativeCase,RDF.type,RDF.Property))
g.add((BM.hasAdministrativeCase,RDFS.domain,BM.LabTestEvent))
g.add((BM.hasAdministrativeCase,RDFS.range,BM.AdministrativeCase))
g.add((BM.hasAdministrativeCase,RDFS.label,Literal("hasLabAdministrativeEvent",lang='en')))

g.add((BM.hasCode,RDF.type,RDF.Property))
g.add((BM.hasCode,RDFS.domain,BM.LabTest))
g.add((BM.hasCode,RDFS.range,RDFS.Literal))
g.add((BM.hasCode,RDFS.label,Literal("hasLabTestCode",lang='en')))

g.add((BM.hasResult,RDF.type,RDF.Property))
g.add((BM.hasResult,RDFS.domain,BM.LabTest))
g.add((BM.hasResult,RDFS.range,BM.LabResult))
g.add((BM.hasResult,RDFS.label,Literal("hasResult",lang='en')))

g.add((BM.hasQuantity,RDF.type,RDF.Property))
g.add((BM.hasQuantity,RDFS.domain,BM.LabResult))
g.add((BM.hasQuantity,RDFS.range,BM.Quantity))
g.add((BM.hasQuantity,RDFS.label,Literal("hasQuantity",lang='en')))

g.add((BM.hasStringValue,RDF.type,RDF.Property))
g.add((BM.hasStringValue,RDFS.domain,BM.LabResult))
g.add((BM.hasStringValue,RDFS.range,XSD.String))
g.add((BM.hasStringValue,RDFS.label,Literal("hasStringValue",lang='en')))

g.add((BM.hasNumericalReference,RDF.type,RDF.Property))
g.add((BM.hasNumericalReference,RDFS.domain,BM.LabResult))
g.add((BM.hasNumericalReference,RDFS.range,BM.ReferenceRange))
g.add((BM.hasNumericalReference,RDFS.label,Literal("hasNumericalReference",lang='en')))

g.add((BM.hasLowerLimit,RDF.type,RDF.Property))
g.add((BM.hasLowerLimit,RDFS.domain,BM.ReferenceRange))
g.add((BM.hasLowerLimit,RDFS.range,BM.Quantity))
g.add((BM.hasLowerLimit,RDFS.label,Literal("hasLowerLimit",lang='en')))

g.add((BM.hasUpperLimit,RDF.type,RDF.Property))
g.add((BM.hasUpperLimit,RDFS.domain,BM.ReferenceRange))
g.add((BM.hasUpperLimit,RDFS.range,BM.Quantity))
g.add((BM.hasUpperLimit,RDFS.label,Literal("hasUpperLimit",lang='en')))

g.add((BM.hasSample,RDF.type,RDF.Property))
g.add((BM.hasSample,RDFS.domain,BM.LabTestEvent))
g.add((BM.hasSample,RDFS.range,BM.Sample))
g.add((BM.hasSample,RDFS.label,Literal("hasSample",lang='en')))

g.add((BM.hasIdentifier,RDF.type,RDF.Property))
g.add((BM.hasIdentifier,RDFS.domain,BM.Sample))
g.add((BM.hasIdentifier,RDFS.range,XSD.String))
g.add((BM.hasIdentifier,RDFS.label,Literal("hasLabIdentifier",lang='en')))


In [ ]:
g.serialize('patient_KG',format='ttl')

### Adding the diagnosis and medicine Ontology

Biomedit Ontology serves the purpose of creating the the framework of the knowedge graph

Now I import a new ontology for medicines which models the high level drugs data. By modelling medicines data, we can exstablish relationship between diagnosis and medicines.

Clinicians/Researchers will then be able to :
1. Reconcile medicines ensuring that medication list is up to date
2. Understand a patients health history more clearly and accordingly treat the patient
3. Make a relation more clearly between a morbidity and a drug


### Medical Subject Headings Ontology

The MESH Ontology is structured such that different drugs(Insulin, Fentanyl) and compounds(Calsium,Patoassium) are in different classes. In order to organize them,   
we define a concept scheme in SKOS

Creating a concept scheme also helps with granularity since each individual abstract concept in MESH has many properties I want to exploit for establishing a semantic relation to the diagnostic concept scheme which helps in predicting diagnosis for patients. For example, the property (http://purl.bioontology.org/ontology/MESH/PA) has an isa relationsip with pharmacological action. the property isa defines the type to be a chemical action.


### Adding data to knowledge graph

#### Assumption - Assuming that insurance status and death nodes are tied to an admission and NOT a patient since insurance status can change and death has a time associated with an admission. In SPHN, Insurance status is a class. I will create a node for insurance status by extending the nomencalture of admission node.

In [ ]:
# Adding patient data

with open('all_data/patients.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)  # Create a reader object
    header = next(reader)  # Read the header row
    print("Header:", header)

    for row_counter, row in enumerate(reader):
        patient_uri = PU[row[0]] # Patient unique identifier is in column 0

        g.add((patient_uri,RDF.type,SDO.Patient))

        g.add((patient_uri,SDO.gender,Literal(row[1],lang='en')))

In [ ]:


with open('all_data/admissions.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)  # Create a reader object
    header = next(reader)  # Read the header row
    print("Header:", header)

    for row_counter, row in enumerate(reader):
        admin_uri = ANU[row[1]] # Admission Event unique identifier is in column 1
        admission_uri = ANU['admission/'+row[1]]
        discharge_uri = ANU['discharge/'+row[1]]
        admission_location_uri = ANU['admission/location'+row[1]]
        discharge_location_uri = ANU['discharge/location'+row[1]]
        insur_uri = ANU['insur/'+row[1]]
        patient_uri = PU[row[0]]  # Patient unique identifier is in column 0

        
        g.add((admin_uri,RDF.type,BM.AdministrativeCase))
        g.add((insur_uri,RDF.type,BM.InsuranceStatus))
        g.add((admission_uri,RDF.type,BM.Admission))
        g.add((admission_location_uri,RDF.type,BM.Location))
        g.add((discharge_uri,RDF.type,BM.Discharge))
        
        
        # linking URIs to other URIs
        g.add((admin_uri,SDO.actor,patient_uri))                           # Linking Patient and Admission event
        g.add((insur_uri,BM.hasAdministrativeCase,admin_uri))
        g.add((admin_uri,BM.hasAdmission,admission_uri))
        g.add((admin_uri,BM.hasDischarge,discharge_uri))
        g.add((discharge_uri,BM.hasTargetLocation,discharge_location_uri))
        g.add((admission_uri,BM.hasTargetLocation,admission_location_uri))

        
        g.add((admission_location_uri,BM.hasExact,Literal(row[6],lang='en')))
        g.add((admission_uri,BM.hasDateTime,Literal(row[2],datatype=XSD.datetime)))
        g.add((discharge_location_uri,BM.hasExact,Literal(row[7],lang='en')))
        g.add((discharge_uri,BM.hasDateTime,Literal(row[3],datatype=XSD.datetime)))
        g.add((insur_uri,BM.hasCoverageType,Literal(row[8],lang='en')))
        g.add((patient_uri,BM.knowsLanguage,Literal(row[0],lang='en')))
        g.add((patient_uri,BM.audienceType,Literal(row[9],lang='en')))
        

In [ ]:
g.serialize('patient_KG',format='ttl')

The URI for each row can be found in pharmacy_id

In [ ]:
from SPARQLWrapper import SPARQLWrapper
sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql", agent="MyRecipeBot/1.0 (your_email@example.com)")  # Apparently adding the agent prevents 403
sparql.setReturnFormat("json")

In [ ]:


def fetch_drug_IRI(drug_label):

    sparql.setQuery(f"""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
    PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
    PREFIX mesh2025: <http://id.nlm.nih.gov/mesh/2025/>
    PREFIX mesh2024: <http://id.nlm.nih.gov/mesh/2024/>
    PREFIX mesh2023: <http://id.nlm.nih.gov/mesh/2023/>

    SELECT ?drugIRI
    FROM <http://id.nlm.nih.gov/mesh>
    WHERE {{
    ?drugIRI rdf:type meshv:Concept ;
    	rdfs:label ?dlabel .
      FILTER (STR(?dlabel) = "{drug_label}")
    }}LIMIT 1
    """)

    
    qres = sparql.query().convert()
    
    try:
        return qres["results"]["bindings"][0]["drugIRI"]["value"]

    except:
        return None

    


In [ ]:
# # Open the prescription CSV file
with open('all_data/prescriptions.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)  # Create a reader object
    header = next(reader)  # Read the header row
    print("Header:", header)

    for row_counter, row in enumerate(reader):
        if row_counter >= 1000:
            break

        else:
            drug_event_uri = DU[row[2]]
            admit_uri = ANU[row[1]]
            drug_uri = DU['drug/'+row[2]]
            substance_uri = DU['drug/substance/'+row[2]]
            
            g.add((drug_event_uri,RDF.type,BM.DrugAdministrationEvent))
            g.add((drug_uri,RDF.type,BM.Drug))
            g.add((substance_uri,RDF.type,BM.Substance))
    
            g.add((drug_event_uri,BM.hasAdministrativeCase,admit_uri))   
            g.add((drug_event_uri,BM.hasDrug,drug_uri))
            g.add((drug_uri,BM.hasActiveIngredient,substance_uri))
            
            g.add((drug_event_uri,BM.hasStartDateTime,Literal(row[6],datatype=XSD.datetime)))
            g.add((drug_event_uri,BM.hasEndDateTime,Literal(row[7],datatype=XSD.datetime)))
    
            
    
            # drug_label = row[9]
            # url = "https://id.nlm.nih.gov/mesh/lookup/descriptor?label="+drug_label+"&match=exact&year=current&limit=10"
            # headers = {
            #     "accept": "application/json"
            # }
            # data = {
            #     "label": drug_label,
            #     "match": "exact",
            #     "limit": "1"
            # }
            
            # response = requests.get(url, headers=headers, json=data)
            response = fetch_drug_IRI(row[9])
            try:
                drug_notation = response.split('/')[-1]
            except:
                drug_notation = None
            
            g.add((substance_uri,BM.hasCode, Literal(drug_notation,datatype=XSD.string) ))
            g.add((substance_uri,BM.hasGenericName,Literal(row[9],datatype=XSD.String)))

In [ ]:
# Open Lab Test CSV file

with open('all_data/labevents.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)  # Create a reader object
    header = next(reader)  # Read the header row
    print("Header:", header)

    for row_counter, row in enumerate(reader):
        lab_event_uri = LU[row[0]]
        lab_uri = LU['lab/'+row[0]]
        admit_uri = ANU[row[2]]
        lab_result_uri = LU['lab/result/'+row[0]]
        quantity_uri = LU['lab/result/quantity/'+row[0]]
        reference_uri = LU['lab/result/reference/'+row[0]]
        reference_upper_quantity_uri = LU['lab/result/reference/upperquantity'+row[0]]
        reference_lower_quantity_uri = LU['lab/result/reference/lowerquantity'+row[0]]
        sample_uri = LU['sample/'+row[4]]


        g.add((lab_event_uri,RDF.type,BM.LabTestEvent))
        g.add((lab_uri,RDF.type,BM.LabTest))
        g.add((admit_uri,RDF.type,BM.AdministrativeCase))
            
        g.add((admit_uri,SDO.workPerformed,lab_event_uri))
        g.add((lab_event_uri,BM.hasAdministrativeCase,admit_uri))
        g.add((lab_event_uri,BM.hasLabTest,lab_uri))
        g.add((lab_uri,BM.hasResult,lab_result_uri))
        g.add((lab_result_uri,BM.hasQuantity,quantity_uri))
        g.add((lab_result_uri,BM.hasNumericalReference,reference_uri))
        g.add((reference_uri,BM.hasUpperLimit,reference_upper_quantity_uri))   
        g.add((reference_uri,BM.hasLowerLimit,reference_lower_quantity_uri))  
        g.add((lab_event_uri,BM.hasSample,sample_uri))
    
        g.add((lab_uri,BM.hasCode,Literal(row[4],datatype=XSD.Double)))
        g.add((quantity_uri,BM.hasValue,Literal(row[7],datatype=XSD.float)))
        g.add((lab_event_uri,BM.hasDateTime,Literal(row[5],datatype=XSD.datetime)))
        g.add((reference_upper_quantity_uri,BM.hasValue,Literal(row[11],datatype=XSD.float)))
        g.add((reference_lower_quantity_uri,BM.hasValue,Literal(row[10],datatype=XSD.float)))


        

In [ ]:
# Open Lab Test key CSV file

with open('all_data/d_labitems.csv', mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)  # Create a reader object
    header = next(reader)  # Read the header row
    print("Header:", header)

    for row_counter, row in enumerate(reader):
        sample_uri = LU['sample/'+row[4]]

        g.add((sample_uri,RDF.type,BM.Sample))

        g.add((sample_uri,BM.hasIdentifier,Literal(row[1],datatype=XSD.String)))

In [ ]:
g.serialize('patient_KG',format='ttl')

In [ ]:
# pip install networkx

In [ ]:
g = Graph()
g.parse("patient_KG.ttl",format="turtle")



In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

nx_graph = nx.DiGraph()
for s, p, o in g:
    nx_graph.add_edge(s.split("#")[-1], o.split("#")[-1], label=p.split("#")[-1])

plt.figure(figsize=(50,40))
pos = nx.spring_layout(nx_graph)  # Layout for visualization
edge_labels = {(u, v): d["label"] for u, v, d in nx_graph.edges(data=True)}

nx.draw(nx_graph, pos, with_labels=True, node_color="lightblue", node_size=3000, edge_color="gray", font_size=10)
nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels=edge_labels, font_size=8)

plt.title("Knowledge Graph Visualization")
plt.show()

### Correctness Metric

This query is used to fetch any incorrectly used properties

In [ ]:
queries ="""
prefix mesh:    <http://purl.bioontology.org/ontology/MESH/>
PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?sub ?obj ?domain ?range ?objectType
WHERE {
    ?sub ?property ?obj .   # General pattern to find subject-object relations

    ?property rdfs:domain ?domain .
    ?property rdfs:range ?range .

    OPTIONAL { ?obj rdf:type ?objectType . }  # Retrieve the object's type (if any)

    FILTER (!BOUND(?objectType) || ?objectType != ?range)  # Ensure type is missing or incorrect
} LIMIT 10
"""

result = g.query(queries)

for row in result:
   print(f"Object: {row['obj']}, Object Type: {row['objectType']}, Range: {row['range']}")

### Consistency metric

In this query, I check the number of patients per admission. which has to be consistent with 1 and not exceed 1

In [ ]:
# SPARQL queries
query ="""
prefix bm:     <http://dbpedia.org/ontology/>
PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs:    <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?event (COUNT(?patient) as ?numPatients)
WHERE {
  ?event sdo:actor ?patient .
}
GROUP BY ?event
HAVING (COUNT(?patient) > 1)


"""

result   = g.query(query)

for row in result:
   print(f"{row['patient']} has {row['numPatient']} runtimes")


### Coherence/Conciseness Metrics

In this query, I find entities without incoming or outgoing relationships

In [ ]:
# SPARQL queries
query ="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bm: <https://biomedit.ch/rdf/sphn-schema/sphn#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?entity
WHERE {
  ?entity a ?obj .
  FILTER NOT EXISTS {
    ?entity ?p ?o .
    FILTER (?p != rdf:type)
  }
}
"""


result   = g.query(query)

for row in result:
   print(f"Entities unused: {row['entity']}")


### Completeness Metrics

In this query I check if all entities are present

In [ ]:
# SPARQL queries
query ="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bm: <https://biomedit.ch/rdf/sphn-schema/sphn#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT (COUNT(?entity) AS ?entityCount) WHERE { ?entity rdf:type/rdfs:subClassOf* ?obj }

"""

result = g.query(query)

for row in result:
   print(f"Typed entities: {row['entityCount']}")


### Schema for Diagnosis

In [ ]:
# Define the schema for diagnosis
g.add((BM.AssessmentEvent,RDF.type,OWL.Class))
g.add((BM.Assessment,RDF.type,OWL.Class))

#Defining the properties
g.add((BM.hasAdministrativeCase,RDF.type,RDF.Property))
g.add((BM.hasAdministrativeCase,RDFS.domain,BM.AssessmentEvent))
g.add((BM.hasAdministrativeCase,RDFS.range,BM.AdministrativeCase))
g.add((BM.hasAdministrativeCase,RDFS.label,Literal("assessmentAdministrativeCase",lang='en')))

g.add((BM.hasAssessment,RDF.type,RDF.Property))
g.add((BM.hasAssessment,RDFS.domain,BM.AssessmentEvent))
g.add((BM.hasAssessment,RDFS.range,BM.Assessment))
g.add((BM.hasAssessment,RDFS.label,Literal("hasAssessment",lang='en')))

g.add((BM.hasName,RDF.type,RDF.Property))
g.add((BM.hasName,RDFS.domain,BM.Assessment))
g.add((BM.hasName,RDFS.range,RDFS.Literal))
g.add((BM.hasName,RDFS.label,Literal("hasName",lang='en')))

g.add((BM.hasCode,RDF.type,RDF.Property))
g.add((BM.hasCode,RDFS.domain,BM.Assessment))
g.add((BM.hasCode,RDFS.range,RDFS.Literal))
g.add((BM.hasCode,RDFS.label,Literal("hasCode",lang='en')))

### Adding Diagnosis to Graph

In [ ]:

for index,row in diagnosis_df.iterrows():
    if row_counter >= 100:
            break
    else:
        # Since diagnosis file does not have a unique identifier, I create my own from a combination of subject_id,
        # admission_id, ICD_code
        diag_uri = DIAGU[f"{row[0]}-{row[1]}-{row[3]}"]
        diag_ass = DIAGU[f"assessment/{row[0]}-{row[1]}-{row[3]}"]
        admin_uri = ANU[row[1]]
         
        g.add((diag_uri,RDF.type,BM.AssessmentEvent))
        
        g.add((diag_uri,BM.hasAdministrativeCase,admin_uri))
        g.add((diag_uri,BM.hasAssessment,diag_ass))
        
        g.add((diag_ass,BM.hasCode,Literal(row[3],datatype=XSD.String)))

### Results

Here I ask query the graph to answer the following questions:
1. For the following ICD codes( code - 41022, 41082, 41042, 41010,41090,) referring to myocardial infarction  fetch the serum levels (lab test codes - 51002,51003,52642) of Troponin, which is a commonly used cardiac enzyme for diagnosis of myocardial infarction
2. How do troponin levels differ between male and female patients diagnosed with MI, and is there a disparity in subsequent treatment decisions?

In [ ]:
## Q1

squery = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bm: <https://biomedit.ch/rdf/sphn-schema/sphn#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?labtestcode ?labvalue ?lowerlimit ?upperlimit
WHERE {

VALUES ?labtestcode { "51002" "51003" "52642" }
VALUES ?icdcode { "41022" "41082" "41042" "41010" "41090" }


?diagass bm:hasCode ?icdCode .
?diaguri bm:hasAssessment ?diagass .
?diaguri bm:hasAdministrativeCase ?adminuri .

?labeventuri bm:hasAdministrativeCase ?adminuri .
?labeventuri bm:hasLabTest ?laburi.
?laburi bm:hasCode ?labtestcode .
?laburi bm:hasResult ?labresulturi.
?labresulturi bm:hasQuantity ?quantityuri.
?quantityuri bm:hasValue ?labvalue.
?labresulturi bm:hasNumericalReference ?referenceuri.
?referenceuri bm:hasUpperLimit ?referenceupp.
?referenceuri bm:hasLowerLimit ?referenceloww.

?quantityuri bm:hasValue ?labvalue.
?referenceloww bm:hasValue ?lowerlimit.
?referenceupp bm:hasValue ?upperlimit.

}
"""

qres = g.query(squery)

for row in qres:
   print(f" Lab Result: {row.labtestcode}  {row.labvalue}  {row.lowerlimit} {row.upperlimit}")

In [ ]:
## Q2

squery = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bm: <https://biomedit.ch/rdf/sphn-schema/sphn#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX sdo: <https://schema.org/>

SELECT ?gender ?labvalue (AVG(?labvalue) AS ?avgTroponin)
WHERE {
  VALUES ?icdcode { "41022" "41082" "41042" "41010" "41090" }
  
  ?diagass bm:hasCode ?icdcode .
  ?diaguri bm:hasAssessment ?diagass .
  ?diaguri bm:hasAdministrativeCase ?adminuri .
  ?adminuri sdo:actor ?patienturi .
  ?patienturi sdo:gender ?gender .

  VALUES ?labtestcode { "51002" "51003" "52642" }
  ?labeventuri bm:hasAdministrativeCase ?adminuri .
  ?labeventuri bm:hasLabTest ?laburi .
  ?laburi bm:hasCode ?labtestcode .
  ?laburi bm:hasResult ?labresulturi .
  ?labresulturi bm:hasQuantity ?quantityuri .
  ?quantityuri bm:hasValue ?labvalue .

}
GROUP BY ?gender
ORDER BY DESC(?avgTroponin)


"""

qres = g.query(squery)

for row in qres:
   print(f" Comparision of troponin values: {row.gender}  {row.labvalue}  {row.avgTroponin}")